# Exercise 5: Lasso Regression on the Franke function  with resampling (Score 10 points))


In [63]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: /home/arefrode/uio/fys-stk4155/projects/FYS-STK4155-Prj1_report


# 1.5
Start off by defining the data

In [64]:
np.random.seed(SEED_VALUE)
maxdegree = 12
degree = 9 # Chosen as the optimal degree from the bias-variance tentative
n = 10000
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))
X = create_X(x,y, n = degree)
z = FrankeFunction(x, y) + 0.2*np.random.normal(0, size = n)

We want to essentially repeat the previous two exercises, as such, this exercise will follow in the footsteps of exercise 4. The difference is that the sci-kit learn implementation of Lasso regression is used. 

In [65]:
nbf_lambdas = 20
lambdas = np.logspace(-4, 1, nbf_lambdas)
z_train_lasso = pd.DataFrame()
z_hat_train_lasso = pd.DataFrame()
z_test_lasso = pd.DataFrame()
z_hat_test_lasso = pd.DataFrame()

Fitting the model for the different values of $\lambda$

In [66]:
# Preparing data and training the model
X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False, random_state=SEED_VALUE)

for lmb in lambdas:
    # Fitting the linear model
    model = Lasso(lmb, fit_intercept=False)
    model.fit(X_train, z_train)

    # Predicting values using the trained model
    z_hat_train = model.predict(X_train)
    z_hat_test = model.predict(X_test)

    # Filling up the dataframes
    z_train_lasso[lmb] = z_train.flatten()
    z_hat_train_lasso[lmb] = z_hat_train.flatten()
    z_test_lasso[lmb] = z_test.flatten()
    z_hat_test_lasso[lmb] = z_hat_test.flatten()

# Calculate the MSE for all lambda values
mse_scores_train = ((z_train_lasso - z_hat_train_lasso) ** 2).mean()
mse_scores_test = ((z_test_lasso - z_hat_test_lasso) ** 2).mean()
# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_lasso - z_hat_train_lasso) ** 2).sum() / ((z_train_lasso - z_train_lasso.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_lasso - z_hat_test_lasso) ** 2).sum() / ((z_test_lasso - z_test_lasso.mean())**2).sum()  



/usr/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 136.86696075119937, tolerance: 0.37641101088929585
  model = cd_fast.enet_coordinate_descent(
/usr/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 94.8286797087087, tolerance: 0.37641101088929585
  model = cd_fast.enet_coordinate_descent(
/usr/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 30.939160541255035, tolerance: 0.37641101088929585
  model = cd_fast.enet_coordinate_descent(
/usr/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. Yo